In [5]:
!pip install streamlit

In [16]:
!ngrok config add-authtoken 34Y49p13TIWZ7WazbTBm1yb9lfe_2GpSGZdYFo2GYaYeNHXKx

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/aerial_project/
!ls

/content/drive/MyDrive/aerial_project
classification_dataset	object_detection_Dataset  results  saved_models


In [9]:
!ls saved_models

best_custom_cnn.h5	   best_efficientnet.h5    history_efficientnet.pkl
best_efficientnet_full.h5  history_custom_cnn.pkl


In [10]:
!ls

classification_dataset	object_detection_Dataset  results  saved_models


In [17]:
%%writefile /content/drive/MyDrive/aerial_project/app.py
#app.py — Streamlit Deployment
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
import io
import os

#Paths
PROJECT_ROOT='/content/drive/MyDrive/aerial_project'
SAVED_MODELS=os.path.join(PROJECT_ROOT, 'saved_models')

#Load models lazily
@st.cache_resource
def load_model(model_name):
    model_path=os.path.join(SAVED_MODELS, model_name)
    model=tf.keras.models.load_model(model_path)
    return model

#Prediction function
def predict_image(model, image, class_names=['bird', 'drone']):
    img=image.resize((224, 224))
    img_array=tf.keras.utils.img_to_array(img)
    img_array=np.expand_dims(img_array, axis=0) / 255.0
    preds=model.predict(img_array)
    pred_class=np.argmax(preds, axis=1)[0]
    confidence=float(np.max(preds))
    return class_names[pred_class].capitalize(), confidence

#Streamlit UI
st.set_page_config(page_title="Aerial Object Classifier", layout="centered")

st.title("Aerial Object Classification (Bird vs Drone)")
st.markdown("This app uses a trained deep learning model to classify aerial images as **Bird** or **Drone**.")

st.sidebar.header("Model Selection")
model_choice = st.sidebar.radio(
    "Select Model:",
    ("Custom CNN (Best Performing)", "EfficientNet (Transfer Learning)")
)

if model_choice == "Custom CNN (Best Performing)":
    model_file = "best_custom_cnn.h5"
else:
    model_file = "best_efficientnet_full.h5"

# Load selected model
with st.spinner(f"Loading {model_choice}..."):
    model = load_model(model_file)
st.success(f"{model_choice} loaded successfully!")

# Image upload section
uploaded_file = st.file_uploader("📤 Upload an aerial image (Bird or Drone):", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file).convert('RGB')
    st.image(image, caption="Uploaded Image", use_container_width=True)

    if st.button("Classify Image"):
        with st.spinner("Analyzing..."):
            pred_class, confidence = predict_image(model, image)
        st.success(f"**Prediction:** {pred_class}")
        st.info(f"**Confidence:** {confidence*100:.2f}%")

        st.markdown("---")
        st.caption("YOLOv8 object detection can be integrated here in the future for bounding box visualization.")
else:
    st.warning("Please upload an image to classify.")

st.markdown("---")
st.markdown("**Project:** Aerial Object Classification & Detection  \n**Domain:** Aerial Surveillance, Wildlife Monitoring, Security & Defense")

Overwriting /content/drive/MyDrive/aerial_project/app.py


In [18]:
from pyngrok import ngrok
import os

# Kill any old ngrok/streamlit processes
!kill $(ps aux | grep streamlit | grep -v grep | awk '{print $2}') 2>/dev/null || true

# Connect ngrok tunnel
public_url = ngrok.connect(8501)
print("🚀 Streamlit URL:", public_url)

# Run Streamlit in background
!streamlit run /content/drive/MyDrive/aerial_project/app.py &>/dev/null &

🚀 Streamlit URL: NgrokTunnel: "https://overdramatically-unshorn-elenor.ngrok-free.dev" -> "http://localhost:8501"


In [14]:
!pip install pyngrok streamlit -q
from pyngrok import ngrok

# Kill any running tunnels
!kill $(ps aux | grep streamlit | grep -v grep | awk '{print $2}') 2>/dev/null || true

# Start Streamlit app
public_url = ngrok.connect(8501)
print("Streamlit URL:", public_url)
!streamlit run /content/drive/MyDrive/aerial_project/app.py &>/dev/null &

ERROR:pyngrok.process.ngrok:t=2025-10-25T07:18:11+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-10-25T07:18:11+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-10-25T07:18:11+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.